In [1]:
%matplotlib inline


# K-NN on the MNIST dataset - PyTorch API


The :mod:`.argKmin(K)` reduction supported by KeOps :class:`pykeops.torch.LazyTensor` allows us
to perform **bruteforce k-nearest neighbors search** with four lines of code.
It can thus be used to implement a **large-scale** 
`K-NN classifier <https://en.wikipedia.org/wiki/K-nearest_neighbors_algorithm>`_,
**without memory overflows** on the 
full `MNIST <http://yann.lecun.com/exdb/mnist/>`_ dataset.


Setup 
-----------------
Standard imports:



In [2]:
import time

import torch
from matplotlib import pyplot as plt

from pykeops.torch import LazyTensor

use_cuda = torch.cuda.is_available()
tensor = torch.cuda.FloatTensor if use_cuda else torch.FloatTensor

Load the MNIST dataset: 70,000 images of shape (28,28).



In [3]:
try:
    from sklearn.datasets import fetch_openml
except ImportError:
    raise ImportError("This tutorial requires Scikit Learn version >= 0.20.")

mnist = fetch_openml('mnist_784', cache=False)

x = tensor( mnist.data.astype('float32') )
y = tensor( mnist.target.astype('int64') )

Split it into a train and test set:



In [4]:
D = x.shape[1]
Ntrain, Ntest = (60000, 10000) if use_cuda else (1000, 100)
x_train, y_train = x[:Ntrain,:], y[:Ntrain]
x_test,  y_test  = x[Ntrain:Ntrain+Ntest,:], y[Ntrain:Ntrain+Ntest]

K-Nearest Neighbors search
----------------------------
Perform the K-NN classification on 10,000 test images in dimension 784:




In [6]:
K = 3  # N.B.: K has very little impact on the running time

start = time.time()    # Benchmark:

X_i = LazyTensor(x_test[:, None, :])  # (10000, 1, 784) test set
X_j = LazyTensor(x_train[None, :, :])  # (1, 60000, 784) train set
D_ij = ((X_i - X_j) ** 2).sum(-1)  # (10000, 60000) symbolic matrix of squared L2 distances

ind_knn = D_ij.argKmin(K, dim=1)  # Samples <-> Dataset, (N_test, K)
lab_knn = y_train[ind_knn]  # (N_test, K) array of integers in [0,9]
y_knn, _ = lab_knn.mode()   # Compute the most likely label

if use_cuda: torch.cuda.synchronize()
end = time.time()

error = (y_knn != y_test).float().mean().item()
time  = end - start

print("{}-NN on the full MNIST dataset: test error = {:.2f}% in {:.2f}s.".format(K, error*100, time))

RuntimeError: [KeOps] This KeOps shared object has been compiled without cuda support: try to set tagHostDevice to 0 or recompile the formula with a working version of cuda.

Fancy display: looks good!



In [ ]:
plt.figure(figsize=(12,8))
for i in range(6):
    ax = plt.subplot(2,3,i+1)
    ax.imshow( (255 - x_test[i]).view(28,28).detach().cpu().numpy(), cmap="gray" )
    ax.set_title( "label = {}".format(y_knn[i].int()) )
    plt.axis('off')

plt.show()

In [27]:
import numpy as np
import pykeops.numpy as pknp

x = np.arange(1, 10).reshape(-1, 3).astype('float32')
y = np.arange(3, 9).reshape(-1, 3).astype('float32')

my_conv = pknp.Genred('SqNorm2(x - y)', ['x = Vi(3)', 'y = Vj(3)'])
print(my_conv(x, y))

Compiling libKeOpsnumpy5ac3d464a2 in /home/rob/.cache/pykeops-1.2-cpython-37//build-libKeOpsnumpy5ac3d464a2:
       formula: Sum_Reduction(SqNorm2(x - y),1)
       aliases: x = Vi(0,3); y = Vj(1,3); 
       dtype  : float64
... Done.


RuntimeError: [KeOps] This KeOps shared object has been compiled without cuda support: 
 1) to perform computations on CPU, simply set tagHostDevice to 0
 2) to perform computations on GPU, please recompile the formula with a working version of cuda.